In [25]:
import pandas as pd
import json
from fuzzywuzzy import process
from mlx_lm import load, generate
import re
import functools

company_names = {
    "ucb": "UCB.BR",
    "abinbev": "ABI.BR",
    "aedifica": "AED.BR",
    "ageas": "AGS.BR",
    "aperam": "APAM.AS",
    "argenx": "ARGX.BR",
    "belgian national bank": "BNB.BR",
    "cofinimmo": "COFB.BR",
    "elia": "ELI.BR",
    "ishares msci belgium etf": "EWK",
    "groupe bruxelles lambert sa": "GBLB.BR",
    "galapagos nv": "GLPG.AS",
    "kbc bank": "KBC.BR",
    "melexis nv": "MELE.BR",
    "proximus": "PROX.BR",
    "sofina": "SOF.BR",
    "solvay": "SOLB.BR",
    "umicore": "UMI.BR",
    "unified post group": "UPG.BR",
    "warehouses de pauw": "WDP.BR",
    "xior student housing": "XIOR.BR",
}

companies = list(company_names.keys())
# Threshold for acceptable similarity (0-100 scale)
threshold = 80
model, tokenizer = load("mlx-community/Llama-3.2-3B-Instruct-4bit")


def find_best_match(user_input, company_list, threshold):
    # Find the best match with score
    best_match, best_score = process.extractOne(user_input, company_list)

    # Check if the best match meets the threshold
    if best_score >= threshold:
        return best_match
    else:
        return "none"


def read_file_to_string(file_name):
    try:
        with open(file_name, "r", encoding="utf-8") as file:
            content = file.read()
        return content
    except FileNotFoundError:
        return "Error: File not found."
    except IOError:
        return "Error: Could not read the file."


def retrieve_stock_etf_info(companies: list, stock_name: str, threshold: int) -> str:
    stock_name = stock_name.lower()
    result = find_best_match(stock_name, companies, threshold)
    # let's do a fuzzy match with the company names and retrieve the most relevant company name
    res = ""
    if result != "none":
        file_content = read_file_to_string("data/" + company_names[result] + ".txt")
        res = json.dumps({"information we have on stock/etf": file_content})
    else:
        res = json.dumps({"error": "transaction id not found."})
    return res


def extract_json(input):

    json_match = re.search(r"\{.*\}", input)
    if json_match:
        json_string = json_match.group()
        data = json.loads(json_string)
        return data
    else:
        return None


def tool_call(message, model, tokenizer):
    message = [
        {
            "role": "user",
            "content": message,
        }
    ]

    # Define tools available for the model to use:
    tool_message = [
        {
            "name": "retrieve_stock_etf_info",
            "description": "get info about a certain stock or etf given the name of the instrument",
            "parameter_definitions": {
                "stock_name": {
                    "description": "Name of the stock or ETF for which we need information",
                    "type": "str",
                    "required": True,
                }
            },
        },
        {
            "name": "directly_answer",
            "description": "Calls a standard (un-augmented) AI chatbot to generate a response given the conversation history",
            "parameter_definitions": {},
        },
    ]
    formatted_input = tokenizer.apply_chat_template(
        message, tools=tool_message, tokenize=False, add_generation_prompt=True
    )
    response = generate(model, tokenizer, prompt=formatted_input, verbose=False)
    json = extract_json(response)
    if json is None or json["name"] == "directly_answer":
        return None
    else:
        names_to_functions = {
            "retrieve_stock_etf_info": functools.partial(
                retrieve_stock_etf_info, companies=companies, threshold=80
            ),
        }

        # to be called after the tool answer
        function_name = json["name"]
        function_params = json["parameters"]
        function_result = names_to_functions[function_name](**function_params)
        return function_result

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

strategy : 
- every message in : 
we apply function call
- if function is retrieved : get result, put in message + message in and output
- if none, put none, and get answer
then iterate

In [28]:
answer = tool_call("what is the stock price of UCB today??", model, tokenizer)

In [29]:
print(answer)

{"information we have on stock/etf": "Yahoo ID of Stock: UCB.BR\nlast traded price: 178.0\ncurrency: EUR\nstock value change since market open: -0.25\nstock value change since market last close: -0.75\nstock exchange on which stock is traded: BRU\nvolume: 115187\nturnover: 20503286.0\nmarket Cap: 33770160128\nnumber of shares outstanding: 189720000\ncompany information in json format: [{'maxAge': 1, 'name': 'Mr. Jean-Christophe  Tellier', 'age': 64, 'title': 'CEO & Executive Director', 'yearBorn': 1959, 'fiscalYear': 2023, 'totalPay': 4099149, 'exercisedValue': 0, 'unexercisedValue': 1781620}, {'maxAge': 1, 'name': 'Ms. Sandrine  Dufour CFA', 'age': 57, 'title': 'Executive VP, CFO & Chief Corporate Development', 'yearBorn': 1966, 'fiscalYear': 2023, 'exercisedValue': 0, 'unexercisedValue': 0}, {'maxAge': 1, 'name': 'Dr. Kirsten  Lund-Jurgensen Ph.D.', 'age': 63, 'title': 'Executive Vice President of Supply & Technology Solutions', 'yearBorn': 1960, 'fiscalYear': 2023, 'exercisedValue':

In [20]:
user_input = "what is the weather like today??"

In [21]:
# Format message with the command-r tool use template
messages = [
    {
        "role": "user",
        "content": user_input,
    }
]
# Define tools available for the model to use:
tools = [
    {
        "name": "retrieve_stock_etf_info",
        "description": "get info about a certain stock or etf given the name of the instrument",
        "parameter_definitions": {
            "stock_name": {
                "description": "Name of the stock or ETF for which we need information",
                "type": "str",
                "required": True,
            }
        },
    },
    {
        "name": "directly_answer",
        "description": "Calls a standard (un-augmented) AI chatbot to generate a response given the conversation history",
        "parameter_definitions": {},
    },
]

formatted_input = tokenizer.apply_chat_template(
    messages, tools=tools, tokenize=False, add_generation_prompt=True
)
response = generate(model, tokenizer, prompt=formatted_input, verbose=False)

In [22]:
print(response)

{"name": "directly_answer", "parameters": {}}<|eom_id|><|start_header_id|>assistant<|end_header_id|>

The function "directly_answer" is used to call a standard (un-augmented) AI chatbot to generate a response given the conversation history. Since there is no conversation history provided, the function will return a default response.


In [23]:
import re

json_match = re.search(r"\{.*\}", response)
if json_match:
    json_string = json_match.group()
    data = json.loads(json_string)
    print(data)
else:
    print("No JSON found")

{'name': 'directly_answer', 'parameters': {}}


In [ ]:
import functools

names_to_functions = {
    "retrieve_stock_etf_info": functools.partial(
        retrieve_stock_etf_info, companies=companies, threshold=80
    ),
}

# to be called after the tool answer
function_name = data["name"]
function_params = data["parameters"]
print("\nfunction_name: ", function_name, "\nfunction_params: ", function_params)

function_result = names_to_functions[function_name](**function_params)
function_result

In [ ]:
formatted_input = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

response = generate(model, tokenizer, prompt=formatted_input, verbose=False)

In [ ]:
print(response)

In [ ]:
messages